## Pytorch Tutorial Reihe

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

from torchvision import datasets, transforms

from IPython.display import clear_output

#### #4 - Das erste eigene Neuronale Netz

In [3]:
# Jedes Netz sollte grob diesen Aufbau haben!

class MeinNetz(nn.Module):
    def __init__(self):
        # jede einzelne Schicht im Model ist eine einzelne Variable hier
        super(MeinNetz, self).__init__()
        self.lin1 = nn.Linear(10, 10) # Linear is eine Fkt. die aufgerufen werden kann
        self.lin2 = nn.Linear(10, 10)
        
    
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = self.lin2(x) # <-- letzte immer ohne eine Aktivierungsfkt. damit sicher ein Output zurück kommt
        return x
        
    
    def num_flat_features(self, x):
        # braucht man damit nn.Module was gescheites ausgibt in der Konsole
        size = x.size()[1:]
        num = 1
        for i in size:
            num *= i
        return num
        
model = MeinNetz()
print(model)

MeinNetz(
  (lin1): Linear(in_features=10, out_features=10, bias=True)
  (lin2): Linear(in_features=10, out_features=10, bias=True)
)


#### #5 - Das Netz füttern

In [4]:
input = Variable(torch.randn(10,10))
print(input)

Variable containing:
 0.2286  1.2073 -1.2697  1.2511  0.9695  1.2869 -0.5863 -2.1078 -1.1088 -1.3281
-1.5167  0.4627 -0.9039  0.2372  0.8171  0.9615  0.2399 -1.0104 -0.0789  0.9825
-0.7906  1.9804 -1.4758  2.2157 -0.6175  0.0839 -0.4624  0.0973 -0.3901 -0.8203
 0.1064  1.1078 -0.2464 -0.0435 -0.8691  0.8513 -1.3851 -0.0106 -0.6182  0.8181
 2.1930  1.3023  0.2176  1.8768 -0.9847  0.4382 -0.8564  0.0061 -0.3072 -0.9173
-2.1286  0.5561  0.5116  0.3655  0.5223  1.6007  0.0558  0.7642 -1.8287 -0.1110
 2.5283 -0.2376 -1.3060  0.7654 -1.2156 -0.1949 -1.1956 -2.5344  0.6591 -0.9512
 1.5505  0.6606 -0.2800  0.5978 -1.1864  1.1231 -0.0647  0.4099 -1.3043  1.7683
-1.3894 -0.1509  0.5991 -0.8540  0.7090 -0.4501  0.6290 -1.7021 -0.1623  0.0355
-0.5412  0.1359  0.6589 -0.2145 -0.4294  0.1719  0.9223 -0.8185  0.8332  0.1493
[torch.FloatTensor of size 10x10]



In [5]:
out = model(input)
print(out)

Variable containing:
-0.0370  0.2835 -0.0014  0.0648  0.0294 -0.2380  0.2325 -0.5592  0.0855  0.0164
-0.1272 -0.1228 -0.0585  0.0432 -0.2186 -0.0425  0.2936 -0.2009  0.0710  0.1004
 0.0436  0.0139 -0.0008 -0.0007 -0.2896 -0.2504  0.3285 -0.3080 -0.0437 -0.0370
-0.2315 -0.1255  0.1466  0.1778 -0.1936  0.0810  0.1996 -0.1474 -0.0159  0.0599
 0.2059  0.1375  0.7084  0.3293 -0.4096 -0.4260  0.2012 -0.1996 -0.0290 -0.3456
-0.0734 -0.1073 -0.3677 -0.2772 -0.3441  0.0108  0.4234  0.0306 -0.2328 -0.1323
-0.4334  0.1542  0.6217  0.3474  0.2127 -0.3321 -0.6054 -0.1135 -0.0822  0.0641
-0.2726 -0.2192 -0.0720  0.0167 -0.1749  0.0005  0.1881 -0.0426 -0.0206  0.0147
-0.3291 -0.0646  0.2830  0.1020 -0.1809  0.1890  0.0644 -0.2313  0.1125  0.2589
-0.3660 -0.2028  0.1508  0.2786 -0.0147 -0.0952 -0.0085 -0.2816  0.2113  0.1432
[torch.FloatTensor of size 10x10]



#### #6 - Das Netz lernen lassen

In [6]:
for i in range(1000):
    x = [1,0,0,0,1,0,0,0,1,1] # input
    input = Variable(torch.Tensor([x for _ in range(10)]))
    # input = input.cuda()
    
    out = model(input)

    x = [0,1,1,1,0,1,1,1,0,0] # target is genau opposite von input, i.e. netz muss lernen jedes bit zu flippen
    target = Variable(torch.Tensor([x for _ in range(10)]))
    # target = target.cuda()
    
    # Kriterum wie unser Fehler berechnet werden soll
    criterion = nn.MSELoss()
    loss = criterion(out, target) # d.h. wir vergleichen das berechnete output mit dem eigentlich Ziel
    clear_output(wait=True)
    print(loss)
    
    model.zero_grad() # im gegensatz zu TensorFlow muss bei pytorch die gradienten immer zurücksetzten pro step --> würde zu größeren Lernrate führen als gewollt, wenn ausgelassen
    loss.backward() # loss über backprop zurückfließen lassen
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    optimizer.step()

Variable containing:
1.00000e-09 *
  3.4044
[torch.FloatTensor of size 1]



## #9 - Handschrifterkennung mit dem MNIST Datensatz

In [7]:
# Download MNIST dataset
kwargs = {}
train_data = torch.utils.data.DataLoader(datasets.MNIST('data', train=True, 
                                                    download=True, 
                                                    transform = transforms.Compose([
                                                        transforms.ToTensor(), 
                                                        transforms.Normalize((0.1307,),(0.3081,)) #? 
                                                    ])),
                                    batch_size=64,
                                    shuffle=True,
                                    **kwargs)

test_data = torch.utils.data.DataLoader(datasets.MNIST('data', train=False,
                                                    transform = transforms.Compose([
                                                        transforms.ToTensor(), 
                                                        transforms.Normalize((0.1307,),(0.3081,))
                                                    ])),
                                    batch_size=64,
                                    shuffle=True,
                                    **kwargs)



#### #10 - Handschrifterkennung mit dem MNIST Datensatz - Training

In [8]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv_dropout = nn.Dropout2d() #?
        self.fc1 = nn.Linear(320, 60)
        self.fc2 = nn.Linear(60, 10)
        
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv_dropout(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

In [11]:
model = Netz()
# model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.8)
def train(epoch):
    model.train()
    for batch_id, (data, target) in enumerate(train_data): # 64 datenpunkte pro iteration, s. batch_size
        # data = data.cuda()
        # target = t_arget.cuda()
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        out = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(out, target)
        loss.backward()
        clear_output(wait=True)
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_id * len(data), len(train_data.dataset),
            100 * batch_id / len(train_data), loss.data[0]
        ))
        
for epoch in range(1, 30):
    train(epoch)

Train Epoch: 29 [29984/60000 (100%)]	Loss: 2.297858


#### #12 - Handschrifterkennung mit dem MNIST Datensatz - Evaluieren

In [ ]:
def test():
    model.eval()
    loss = 0
    correct = 0
    for data. target in test_data:
        data = Variable(data, volatile=True)
        target = Variable(target)
        out = model(data)
        loss = nn.CrossEntropyLoss()(out, target)
        
        